In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import torch
import numpy as np
from squaredTopology import *
from feedForward import *
torch.set_default_dtype(torch.float64)
import numdifftools as nd

In [32]:
xref = np.array([1e-9, 1e-9, 1e-9])

def mse(x,y):
    x.requires_grad_(True)
    y.requires_grad_(True)
    mse = torch.sum((torch.abs(x - y))**2)
    mse.requires_grad_(True)
    return mse

def mseXREF(x):
    return np.sum((np.abs(x - xref))**2)

def eval_mse(net, xtest, ytest):
    net.eval()
    cumulative_mse = 0.
    for x, L in zip(xtest, ytest):
        N = net(x)
        cumulative_mse += mse(L, N)
    return cumulative_mse / xtest.shape[0] 

def jacobian(y, x, create_graph=False):                                                               
    jac = []                                                                                          
    flat_y = y.reshape(-1)                                                                            
    grad_y = torch.zeros_like(flat_y)                                                                 
    for i in range(len(flat_y)):                                                                      
        grad_y[i] = 1.                                                                                
        grad_x, = torch.autograd.grad(flat_y, x, grad_y, 
                                      retain_graph=True,
                                      create_graph=create_graph )
        jac.append(grad_x.reshape(x.shape))                                                           
        grad_y[i] = 0.                                                                                
    return torch.stack(jac).reshape(y.shape + x.shape)                                                
                                                                                                      
def hessian(y, x):                                                                                    
    return jacobian(jacobian(y, x, create_graph=True), x)

In [7]:
a = xTest[0]
b = torch.FloatTensor([1e-9, 1e-9, 1e-9])
a.requires_grad_(True)
b.requires_grad_(True)

tensor([1.0000e-09, 1.0000e-09, 1.0000e-09], dtype=torch.float32,
       requires_grad=True)

In [8]:
jacobian(mse(a,b ), a)

tensor([ 1.0839, -3.0856,  0.8529], dtype=torch.float32)

In [9]:
jacobian(sNet(b), b)

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [33]:
def generateData(size = 1000):
    reference = torch.FloatTensor([1e-9, 1e-9, 1e-9])
    xy = torch.FloatTensor(size, 2).uniform_(-2.1, 2.1)
    z = torch.FloatTensor(size, 1).uniform_(0., 1)
    xtrain = torch.cat((xy, z), dim=-1) #VSTACK
    ytrain = torch.stack([mse(x, reference) for x in xtrain], 0).reshape(size, 1) ## Append to torch tensor
      
    print(xtrain.shape)
    print(ytrain.size())

    return xtrain, ytrain
    

In [34]:
xTrain, yTrain = generateData(size = 1000)
xTest, yTest = generateData(10000)

torch.Size([1000, 3])
torch.Size([1000, 1])
torch.Size([10000, 3])
torch.Size([10000, 1])


In [35]:
sNet = trainNetwork(SquaredNet(3, 1), xTrain, yTrain, epochs=5)
print("Evaluted MSE for Squared Topology : ", eval_mse(sNet, xTest, yTest))

  0%|          | 0/5 [00:00<?, ?it/s]


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [14]:
sNet.eval()

SquaredNet(
  (fc1): Linear(in_features=3, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=3, bias=True)
  (fc4): Linear(in_features=1, out_features=1, bias=True)
)

In [16]:
a = xTest[0].requires_grad_(True)
b = sNet(a)
print(b)

tensor([1.0640], dtype=torch.float32, grad_fn=<AddBackward0>)


In [19]:
def jaco(x, y):
    nx = len(x)
    ny = len(y)
    Jy_x = torch.zeros((ny, nx))
    for i in range(ny):
        if x.grad is not None:
            x.grad.data.zero_()
        unit_i = torch.zeros(ny)
        unit_i[i] = 1.
        y.backward(unit_i, retain_graph=True)
        Jy_x[i, :] = x.grad.data
    print(Jy_x)
    return Jy_x

In [20]:
jaco(a, b)

TypeError: can't assign a NoneType to a torch.DoubleTensor

In [21]:
nx = len(a)
ny = len(b)

In [23]:
ny

1

In [24]:
Jy_x = torch.zeros((ny, nx))

In [25]:
Jy_x

tensor([[0., 0., 0.]])

In [26]:
x.grad

tensor([4., 5.])

In [29]:
unit_i = torch.zeros(ny)
unit_i[i] = 1.
b.backward(unit_i, retain_graph=True)


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.